# Load libraries & helper functions

In [1]:
from helpers import *
%matplotlib inline

# Declare parameters

In [2]:
config.DATADIR                 = '/home/duboisjx/mnt10/LabUsers/duboisjx/data/Conte/MRI'
config.pipelineName            = 'preGLM0'
# fMRI runs
fmriRuns                       = ['movie_1','movie_2','LOI1_1','LOI2_1','socnsLOI1_1','socnsLOI2_1']
# use volume or surface data
config.useNative               = False
config.fmriFileTemplate        = '#fMRIrun#_Atlas.dtseries.nii'
# config.fmriFileTemplate        = '#fMRIrun#.nii.gz'
config.melodicFolder           = op.join('#fMRIrun#_hp2000.ica','filtered_func_data.ica') #the code #fMRIrun# will be replaced
config.movementRelativeRMSFile = 'Movement_RelativeRMS.txt'
config.movementRegressorsFile  = 'Movement_Regressors_dt.txt'

tmp = config.fmriFileTemplate.split('.')
if tmp[1]=='nii':
    config.isCifti = False
    config.parcellationName        = 'shen2013'
    config.parcellationFile        = '/home/duboisjx/mnt10/LabUsers/duboisjx/data/parcellations/shenetal_neuroimage2013_new/shen_2.5mm_268_parcellation.nii.gz'
    config.nParcels                = 268
elif tmp[1]=='dtseries':
    config.isCifti = True
    config.parcellationName        = 'Glasser'
    config.parcellationFile        = '/home/duboisjx/mnt10/LabUsers/duboisjx/data/parcellations/Glasser2016/Parcels.dlabel.nii'
    config.nParcels                = 360
else:
    print 'unknown file extension'

config.preWhitening            = False
config.maskParcelswithAll      = True
config.maskParcelswithGM       = False
if config.maskParcelswithGM:
    config.parcellationName    = config.parcellationName + '_GM'

# submit jobs with sge
config.queue        = False
launchSubproc       = True
parallelEnvironment = 'smp' #'openmp'
    
subjects  = [s for s in sorted(os.listdir(config.DATADIR)) if len(s)==6 and s[0:2]=='CC']
print subjects

['CC0003', 'CC0005', 'CC0006', 'CC0007', 'CC0008', 'CC0010', 'CC0011', 'CC0012', 'CC0014', 'CC0024', 'CC0025', 'CC0027', 'CC0029', 'CC0033', 'CC0041', 'CC0046', 'CC0049', 'CC0056', 'CC0058', 'CC0059', 'CC0060', 'CC0061', 'CC0063', 'CC0064', 'CC0065', 'CC0068', 'CC0069', 'CC0070', 'CC0071', 'CC0074', 'CC0076', 'CC0077', 'CC0078', 'CC0079', 'CC0080', 'CC0081', 'CC0082', 'CC0085', 'CC0087', 'CC0088', 'CC0089', 'CC0091', 'CC0092', 'CC0096', 'CC0100', 'CC0101', 'CC0104', 'CC0105', 'CC0106', 'CC0108', 'CC0110', 'CC0111', 'CC0112', 'CC0114', 'CC0115', 'CC0116', 'CC0117', 'CC0118', 'CC0124', 'CC0126']


# Do work

In [3]:
config.overwrite               = False
config.Operations              = config.operationDict[config.pipelineName]

if config.pipelineName in ['SiegelA','SiegelB']:
    config.useFIX              = True
else:
    config.useFIX              = False

returnhere = os.getcwd()

# do the work
config.joblist = list()
iSub    = 0
for subject in subjects:
    config.subject = str(subject)
    iRun = 0
    for config.fmriRun in fmriRuns:
        runPipelinePar(launchSubproc=launchSubproc)
        iRun=iRun+1
        if len(config.joblist)>=8:
            # wait until all tasks are done
            checkProgress(verbose=True)
            config.joblist = list()
    iSub = iSub + 1

if len(config.joblist)>0:
    checkProgress(verbose=True) 
    
# make symlinks
keepSub = np.zeros((len(subjects),len(fmriRuns)),dtype=np.bool_)
iSub    = 0
for subject in subjects:
    config.subject = str(subject)
    iRun = 0
    for config.fmriRun in fmriRuns:
        keepSub[iSub,iRun] = runPipelinePar(launchSubproc=False)
        if keepSub[iSub,iRun]:
            os.chdir(op.dirname(buildpath()))
            if op.isdir(buildpath()+'_'+config.pipelineName):
                remove(buildpath()+'_'+config.pipelineName)
            os.symlink(op.basename(buildpath()),
                op.basename(buildpath())+'_'+config.pipelineName)
            if op.exists(op.join(buildpath(),config.fmriRun+'_'+config.pipelineName+config.ext)):
                remove(op.join(buildpath(),config.fmriRun+'_'+config.pipelineName+config.ext))
            if op.exists(op.join(buildpath(),config.fmriRun+'_'+config.pipelineName+'_Atlas'+config.ext)):
                remove(op.join(buildpath(),config.fmriRun+'_'+config.pipelineName+'_Atlas'+config.ext))
            os.chdir(buildpath())
            os.symlink(op.basename(config.fmriFile_dn),
                  config.fmriRun+'_'+config.pipelineName+'_Atlas'+config.ext)
            print '{} -- {} -- OK'.format(config.subject,config.fmriRun)
            os.chdir(returnhere)
        iRun=iRun+1
    iSub = iSub + 1

submitted sCC0003_movie_1_preGLM0_ciftiTrue_20180511151514388
submitted sCC0003_movie_2_preGLM0_ciftiTrue_20180511151515132
submitted sCC0006_movie_1_preGLM0_ciftiTrue_20180511151516434
submitted sCC0006_movie_2_preGLM0_ciftiTrue_20180511151517411
submitted sCC0006_LOI2_1_preGLM0_ciftiTrue_20180511151518191
submitted sCC0006_socnsLOI1_1_preGLM0_ciftiTrue_20180511151518937
submitted sCC0006_socnsLOI2_1_preGLM0_ciftiTrue_20180511151519411
submitted sCC0007_movie_1_preGLM0_ciftiTrue_20180511151520665
Waiting for 8 jobs to complete...
Waiting for 8 jobs to complete...
Waiting for 8 jobs to complete...
Waiting for 8 jobs to complete...
Waiting for 4 jobs to complete...
Waiting for 4 jobs to complete...
Waiting for 4 jobs to complete...
Waiting for 3 jobs to complete...
Waiting for 3 jobs to complete...
Waiting for 2 jobs to complete...
Waiting for 1 jobs to complete...
Waiting for 1 jobs to complete...
Waiting for 1 jobs to complete...
Waiting for 1 jobs to complete...
All done!!
submitted 